### 1. Preparación del entorno

In [6]:
# Librerías internas (Python standard library)
import os              # trabajar con el sistema operativo y variables de entorno
import glob            # manejo de archivos y rutas

# Librerías externas (third-party)
import pandas as pd    # dataframes para manipular el CSV
import numpy as np     # manejo de valores nulos (NaN)
import mysql.connector # conector oficial de MySQL para Python
from dotenv import load_dotenv  # cargar variables de entorno desde el archivo .env

# Carga de variables de entorno
load_dotenv()  # Devuelve True si se cargan correctamente, False si no

# Variables de entorno
host = os.getenv("MYSQL_HOST")        # Host de la base de datos
user = os.getenv("MYSQL_USER")        # Usuario de la base de datos
password = os.getenv("MYSQL_PASSWORD")# Contraseña de la base de datos

### 2. Creacion BBDD

In [7]:
# --- Intentamos conectarnos al servidor MySQL ---
try:
    # Creamos la conexión y la guardamos en la variable 'cnx'
    cnx = mysql.connector.connect(
        host=os.getenv('MYSQL_HOST'),                  # Dirección del servidor MySQL (localhost o IP)
        user=os.getenv('MYSQL_USER'),                  # Usuario que se conectará (ej. root)
        password=os.getenv('MYSQL_PASSWORD'),          # Contraseña del usuario
        auth_plugin='mysql_native_password',           # Evita errores de conexión por `caching_sha2_password`
        use_pure=True,                                 # Usa el conector puro de Python, más compatible
    )

    # Si llegamos aquí, la conexión fue exitosa
    print('Conexión exitosa')  # Mensaje de confirmación

# --- Capturamos cualquier error relacionado con MySQL ---
except mysql.connector.Error as e:
    # Si ocurre un error, lo imprimimos para diagnosticarlo
    print('Error al conectar:', e)


Conexión exitosa


In [8]:
# --- Crear un cursor ---
# Un cursor es un objeto que permite ejecutar consultas SQL sobre la conexión
mycursor = cnx.cursor()  

# --- Crear la base de datos si no existe ---
# "IF NOT EXISTS" evita errores si la base de datos ya está creada
mycursor.execute("CREATE DATABASE IF NOT EXISTS MusicStream")

# --- Seleccionar la base de datos para usarla ---
# Todas las consultas siguientes se ejecutarán en esta base de datos
mycursor.execute("USE MusicStream")

# --- CREACIÓN DE TABLAS (Normalización de datos) ---
# Tabla 1: Artistas
# Guardamos información biográfica para no repetirla en cada canción
mycursor.execute('''
CREATE TABLE IF NOT EXISTS Artistas (
    Nombre_Artista VARCHAR(255) PRIMARY KEY,  -- Clave primaria: identifica cada artista
    Bio_Resumen TEXT,                         -- Biografía del artista (texto largo)
    Listeners INT                              -- Cantidad de oyentes
);''')

# Tabla 2: Canciones
# Cada canción se relaciona con un artista mediante clave foránea
mycursor.execute('''
CREATE TABLE IF NOT EXISTS Canciones (
    ID_Cancion INT AUTO_INCREMENT PRIMARY KEY, -- Identificador único y automático
    Titulo TEXT NOT NULL,                      -- Título de la canción (no puede ser nulo)
    Album TEXT,                                -- Nombre del álbum
    Genero VARCHAR(100),                       -- Género musical
    Anio YEAR,                                 -- Año de lanzamiento
    Artista_Relacion VARCHAR(255),            -- Columna que vincula con la tabla Artistas
    FOREIGN KEY (Artista_Relacion) REFERENCES Artistas(Nombre_Artista)  -- Relación entre tablas
);''')

# --- Confirmación visual de éxito ---
print("✅ Estructura de tablas (Relacional) creada correctamente.")


✅ Estructura de tablas (Relacional) creada correctamente.


In [9]:
# -------------------------------
# 4️⃣ DETECCIÓN DE CSVs Y PROCESO
# -------------------------------
archivos = glob.glob("canciones_*.csv")  # Todos los CSV que cumplan el patrón

for archivo in archivos:
    print(f"\n📂 Procesando archivo: {archivo}")
    
    # Leer CSV
    df = pd.read_csv(archivo, sep=';', encoding='utf-8-sig')
    
    # Limpiar valores nulos
    df_limpio = df.replace({np.nan: None, 'nan': None, 'NaN': None})
    
    # Limpiar espacios extra en nombres de artistas
    if 'artista' in df_limpio.columns:
        df_limpio['artista'] = df_limpio['artista'].str.strip()
    
    try:
        # --- PASO A: INSERTAR ARTISTAS ---
        df_artistas = df_limpio[['artista', 'bio_resumen', 'listeners']].drop_duplicates(subset=['artista'])
        lista_artistas = list(df_artistas.itertuples(index=False, name=None))
        query_artistas = "INSERT IGNORE INTO Artistas (Nombre_Artista, Bio_Resumen, Listeners) VALUES (%s, %s, %s)"
        mycursor.executemany(query_artistas, lista_artistas)
        print(f"📦 {len(df_artistas)} artistas insertados o ignorados si existían")

        # --- PASO B: INSERTAR CANCIONES ---
        df_canciones = df_limpio[['nombre', 'album', 'genero', 'año', 'artista']]
        lista_canciones = list(df_canciones.itertuples(index=False, name=None))
        query_canciones = "INSERT INTO Canciones (Titulo, Album, Genero, Anio, Artista_Relacion) VALUES (%s, %s, %s, %s, %s)"
        mycursor.executemany(query_canciones, lista_canciones)
        print(f"🎵 {len(df_canciones)} canciones insertadas")

        # Guardar cambios
        cnx.commit()
        print("🚀 Archivo procesado con éxito")
    
    except mysql.connector.Error as e:
        print(f"❌ Error en {archivo}: {e}")
        cnx.rollback()



📂 Procesando archivo: canciones_2010_con_lastfm.csv
📦 96 artistas insertados o ignorados si existían
🎵 11039 canciones insertadas
🚀 Archivo procesado con éxito

📂 Procesando archivo: canciones_2012_con_lastfm.csv
📦 118 artistas insertados o ignorados si existían
🎵 13811 canciones insertadas
🚀 Archivo procesado con éxito

📂 Procesando archivo: canciones_2014_con_lastfm.csv
📦 121 artistas insertados o ignorados si existían
🎵 13230 canciones insertadas
🚀 Archivo procesado con éxito

📂 Procesando archivo: canciones_2016_con_lastfm.csv
📦 128 artistas insertados o ignorados si existían
🎵 9264 canciones insertadas
🚀 Archivo procesado con éxito

📂 Procesando archivo: canciones_2018_con_lastfm.csv
📦 150 artistas insertados o ignorados si existían
🎵 10335 canciones insertadas
🚀 Archivo procesado con éxito

📂 Procesando archivo: canciones_2020_con_lastfm.csv
📦 168 artistas insertados o ignorados si existían
🎵 12156 canciones insertadas
🚀 Archivo procesado con éxito
